In [1]:
## (Temp) Install packages
install.packages('devtools')
install.packages('latex2exp')
library(devtools)
install_github('susanathey/MCPanel')
install.packages('rjson')

Installing package into 'C:/Users/lzl25/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'devtools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lzl25\AppData\Local\Temp\RtmpW8x4Aq\downloaded_packages


Installing package into 'C:/Users/lzl25/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'latex2exp' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lzl25\AppData\Local\Temp\RtmpW8x4Aq\downloaded_packages


Warning message:
"package 'devtools' was built under R version 3.6.3"Loading required package: usethis
Skipping install of 'MCPanel' from a github remote, the SHA1 (6b2706fd) has not changed since last install.
  Use `force = TRUE` to force installation
Installing package into 'C:/Users/lzl25/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'rjson' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rjson'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"拷贝C:\Users\lzl25\Documents\R\win-library\3.6\00LOCK\rjson\libs\x64\rjson.dll到C:\Users\lzl25\Documents\R\win-library\3.6\rjson\libs\x64\rjson.dll时出了问题：Permission denied "Warning message:
"restored 'rjson'"


The downloaded binary packages are in
	C:\Users\lzl25\AppData\Local\Temp\RtmpW8x4Aq\downloaded_packages


## Loading source files

To implement matrix completion method R package [susanathey/MCPanel](https://github.com/susanathey/MCPanel) is used.

In [2]:
library(MCPanel)
library(glue)
library(gtools)
library(rjson)

## Check which senario we are using

In [3]:
senario <- fromJSON(file = "src/senario_selection.json")$senario
T0 <- fromJSON(file = "src/senario_selection.json")$T0
print(glue('We are using senario ', senario, ' with T0 = {T0}'))

We are using senario D with T0 = 24


## Loading data

In [4]:
# Path to control and true untreated data
path_control <- glue('src/Senario{senario}/control/')
path_untreat <- glue('src/Senario{senario}/untreat/')

# List the filenames for data
filenames_control <- mixedsort(list.files(path = path_control))
filenames_untreat <- mixedsort(list.files(path = path_untreat))

# Loading data
control_data <- list()
for(i in 1:length(filenames_control)){
  data <- read.csv(paste(path_control,filenames_control[i], sep = ''))
  control_data[[i]] <- data
}

untreat_data <- list()
for(i in 1:length(filenames_untreat)){
  data <- read.csv(paste(path_untreat,filenames_untreat[i], sep = ''))
  untreat_data[[i]] <- data
}

## Estimating counterfactual outcomes using MC method

In [5]:
# Combine control and treated units to form a complete matrix.
comb_data <- list()
for(i in 1:length(filenames_control)){
  data_con <- control_data[[i]]
  data_tre <- t(untreat_data[[i]])
  colnames(data_tre) <- colnames(data_con)
  comb_data[[i]] <- rbind(data_con, data_tre)
}

In [6]:
# Implement MC.
for(i in 1:length(comb_data)){
  print(glue("{i}th implementation starts"))
  flush.console()
  data <- comb_data[[i]]
  N <- nrow(data)
  T <- ncol(data)
  treat_mat <- matrix(1, N, T)
  treat_mat[N, (T0 + 1):T] <- 0
  data_obs <- data * treat_mat
  
  est_model_MCPanel <- mcnnm_cv(as.matrix(data_obs), treat_mat, to_estimate_u = 1, to_estimate_v = 1)
  Mhat <- est_model_MCPanel$L + replicate(T,est_model_MCPanel$u) + t(replicate(N,est_model_MCPanel$v))
  
  file_mc = glue('mc_imputed_{i}')
  write.table(Mhat, file=glue('base_learners/mc_counterfactual/{file_mc}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  
}

1th implementation starts
2th implementation starts
3th implementation starts
4th implementation starts
5th implementation starts
6th implementation starts
7th implementation starts
8th implementation starts
9th implementation starts
10th implementation starts
11th implementation starts
12th implementation starts
13th implementation starts
14th implementation starts
15th implementation starts
16th implementation starts
17th implementation starts
18th implementation starts
19th implementation starts
20th implementation starts
21th implementation starts
22th implementation starts
23th implementation starts
24th implementation starts
25th implementation starts
26th implementation starts
27th implementation starts
28th implementation starts
29th implementation starts
30th implementation starts
31th implementation starts
32th implementation starts
33th implementation starts
34th implementation starts
35th implementation starts
36th implementation starts
37th implementation starts
38th imple

Click here to go to [03 Estimate counterfactual outcomes for each base learner](03%20Estimate%20counterfactual%20outcomes%20for%20each%20base%20learner.ipynb)